# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
data = 'Data/API_data.csv'

df = pd.read_csv(data)

df = df.dropna()

for_map_locations = df[["Latitude", "Longitude"]]

humidity = df['Humidity'].tolist()

print(len(locations))
print(len(humidity))

df.sample(15)


568
568


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind_Speed,Country
18,Tsarítsani,39.88,22.23,10.00,93,75,1.01,GR
425,Manama,26.22,50.58,18.00,55,0,4.10,BH
449,De-Kastri,51.47,140.78,-25.89,79,67,2.67,RU
19,Kapaa,22.08,-159.32,23.00,94,90,10.30,US
541,Unaí,-16.36,-46.91,26.77,73,40,1.16,BR
144,Nadym,65.53,72.52,-39.02,81,6,3.93,RU
304,Bushehr,28.97,50.84,13.00,71,0,4.10,IR
215,Deogarh,21.53,84.73,13.57,46,0,1.01,IN
260,Nishihara,26.18,127.76,18.89,72,20,4.10,JP
371,Nagorsk,59.32,50.81,-9.77,95,100,4.05,RU



### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(for_map_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
criteria_df = df.loc[((df['Max Temp'] > 36) & (df['Humidity'] < 50) & (df['Cloudiness'] < 50) & (df['Wind_Speed'] < 4))]
criteria_df



,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind_Speed,Country
128,General Roca,-39.03,-67.58,37.0,27,20,3.1,AR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
criteria_df['Hotel Name'] = ""
criteria_df

locations_df = criteria_df[["Latitude", "Longitude"]].astype(str)

locations_df["LatLng"] = criteria_df["Latitude"].astype(str) + ", " + criteria_df["Longitude"].astype(str)
locations_df

locations_list = locations_df['LatLng'].tolist()
locations_list

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_radius = 5000

locations_list

['-39.03, -67.58']

In [14]:
for location in locations_list:
    
    params = {
    "location": location,  
    "rankby": "distance",
    "type": "hotel",
    "radius" : target_radius,    
    "key": g_key,
}
    
    response = requests.get(base_url, params=params).json()
    
print(json.dumps(response, indent=4, sort_keys=True))


{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
